In [1]:
import sys
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import os
from astropy.cosmology import Planck15 as cosmo #Planck 2015
from astropy.cosmology import z_at_value

import get_ZdepSFRD as Z_SFRD
import importlib
import paths
import requests
import illustris_python as il
import illustris_python.groupcat as gc
import illustris_python.snapshot as sn

In [2]:
TNG  = 100
lvl = 1
TNGpath = "/home/sashalvna/research/TNGdata/TNG%s" % (TNG)
baseUrl = 'http://www.tng-project.org/api/'
headers = {"api-key":"e36226423a0cc5e62f2e553f39b44238"}

In [3]:
def get(path, params=None):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically
    return r

In [4]:
#get basic snapshots information (redshift, url) for each of the 100 snapshots

r = get(baseUrl) #path to all Illustris TNG simulations
names = [sim['name'] for sim in r['simulations']] #names of all available simulations
i = names.index('TNG%s-%s' % (TNG, lvl)) #index of specific simulation
sim = get( r['simulations'][i]['url'] ) #get path to that simulation
snaps = get( sim['snapshots'] ) #get path to snapshots for that simulation
nfiles = sim['num_files_snapshot'] #number files per snapshot

In [5]:
os.system("wget -nd -nc -nv -e robots=off -l 1 -r -A hdf5 --content-disposition --header='API-Key: e36226423a0cc5e62f2e553f39b44238' 'http://www.tng-project.org/api/TNG100-1/files/snapshot-099.447.hdf5?gas=GFM_Metallicity,StarFormationRate'")

0

In [6]:
def get_TNGsnapcols(TNGpath, TNG, lvl, snap, nfiles, cols='gas=GFM_Metallicity,StarFormationRate'):
    #downloads a single snapshot subfile; requires path to local TNG data files, which TNG and which level (e.g TNG100-"1"), 
    #which snapshot, which subfile in snapshot, and which data columns from snapshot
    
    #check if snapshot subfile already exists; if not, download
    if not os.path.exists(TNGpath + "/%03d" %(snap)):
        os.chdir(TNGpath)
        os.mkdir("%03d" %(snap))
        
    os.chdir(TNGpath + "/%03d" %(snap)) #go to correct directory before downloading
    for ifile in range(nfiles):
        fname = TNGpath + "/%03d/snap_%03d.%s.hdf5" %(snap, snap, ifile)    
        if not os.path.isfile(fname):
            os.system("wget -nd -nc -nv -e robots=off -l 1 -r -A hdf5 --content-disposition --header='API-Key: e36226423a0cc5e62f2e553f39b44238' 'http://www.tng-project.org/api/TNG%s-%s/files/snapshot-%03d.%s.hdf5?%s'" %(TNG, lvl, snap, ifile, cols)) 

In [7]:
def delete_snapshotsubfile(TNGpath, snap, ifile):
    #deletes a single snapshot; requires requires path to local TNG data files, 
    #which snapshot, which file in snapshot, and which data columns from snapshot
    
    #check if snapshot subfile exists; if yes, delete
    fname = TNGpath + "/%03d/snap_%03d.%s.hdf5" %(snap, snap, ifile)
    if os.path.exists(fname):
        os.remove(fname)

In [11]:
def get_z_and_lookbacktime(snaps):
    #get the redshifts for each snapshot
    redshifts = np.array([])
    for ind, val in enumerate(snaps):
        redshifts = np.append(redshifts, snaps[ind]['redshift'])
    
    #calculate lookback time for each snapshot
    lookbacktimes = np.array(cosmo.lookback_time(redshifts))
    
    return redshifts, lookbacktimes

In [12]:
#function to get binned SFR for each snapshot
def getSFRMetallicityFromGas(TNGpath, TNG, lvl, snap, nfiles, nBinsSFR=60):
    mbins = np.logspace(-10, 0., nBinsSFR+1)
    sfrs  = np.zeros((nBinsSFR))
    
    fname =  TNGpath + "/%03d/snap_%03d.%s.hdf5"%(snap, snap, "%d") 
    get_TNGsnapcols(TNGpath, TNG, lvl, snap, nfiles)
    
    for ifile in range(nfiles):
        
        with h5.File(fname % ifile, "r") as f:
      
            pStars = f["PartType0"]
      
            SFR = pStars["StarFormationRate"][:]
            Metals = pStars["GFM_Metallicity"][:]
      
            data, e = np.histogram(Metals, bins=mbins, weights=SFR)
            sfrs += data
            
            delete_snapshotsubfile(TNGpath, snap, ifile) #delete subfile after getting data from it 
            
    return sfrs

In [13]:
#function to make the data file; iterates through all snapshots

def getFullSFRMetallicityFromGas(TNGpath, TNG, lvl, snaps, nfiles, nbins=60):
    outfname = TNGpath + "/SFRMetallicityFromGasTNG%d-%d.hdf5" % (TNG,lvl)
    sfrs = np.zeros((len(snaps),nbins))
    redshifts, lookbacktimes = get_z_and_lookbacktime(snaps)
  
    Count = 0
    for snap in range(2): #range(len(snaps)):
        if sfrs[snap].sum() == 0:
            print( "Doing snap %d." % snap )
            s = getSFRMetallicityFromGas(TNGpath, TNG, lvl, snap, nfiles, nBinsSFR=nbins)
            sfrs[snap,:] = s
            Count += 1

    if Count > 0:
        mbins = np.logspace( -10, 0., nbins+1 )
        with h5.File(outfname, "a") as f:
            f.create_dataset('MetalBins', data=mbins)
            f.create_dataset('Redshifts', data=redshifts)
            f.create_dataset('Lookbacktimes', data=lookbacktimes)
            f.create_dataset('Sfr', data=sfrs)

In [10]:
nfiles_temp = 6
getFullSFRMetallicityFromGas(TNGpath, TNG, lvl, snaps, nfiles_temp, nbins=60)

In [16]:
TNGfilename = "SFRMetallicityFromGasTNG100-1.hdf5"
with h5.File(TNGpath + "/" + TNGfilename, "r") as f:
    MetalBins         = f["MetalBins"][:]
    Obs_Lookbacktimes = f["Lookbacktimes"][:]
    BoxSfr            = f["Sfr"][:]
    Redshifts         = f["Redshifts"][:]

In [17]:
sumsfr = np.sum(BoxSfr, axis=1)

In [18]:
len(sumsfr[sumsfr != 0])

2

In [19]:
BoxSfr[4]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [20]:
data_dir    =  str(paths.data)
TNGfilename = '/SFRMetallicityFromGasTNG100-1.hdf5'
with h5.File(data_dir+TNGfilename, "r") as f:
    MetalBins         = f["MetalBins"][:]
    Obs_Lookbacktimes = f["Lookbacktimes"][:]
    BoxSfr            = f["Sfr"][:]
    Redshifts         = f["Redshifts"][:]

In [21]:
BoxSfr[0]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       4.75083245e-04, 3.65766156e+00, 1.04157008e-02, 5.71727753e-04,
       6.40153885e-04, 0.00000000e+00, 5.22613525e-04, 4.58240509e-04,
       0.00000000e+00, 5.95665770e-04, 1.04798097e-03, 3.41892242e-04,
       3.94334784e-04, 1.79171562e-03, 0.00000000e+00, 0.00000000e+00,
       4.54902649e-04, 1.68185338e-03, 3.15117836e-03, 1.67511031e-03,
       1.68609619e-03, 1.07360142e-03, 2.74735253e-03, 1.85179710e-03,
       0.00000000e+00, 1.97128486e-03, 2.81886768e-03, 1.78170204e-03,
       1.47175789e-03, 5.59481466e-03, 6.16976223e-03, 1.28049477e-02,
       7.56591576e-03, 2.91800499e-03, 8.31058109e-03, 2.18614138e-02,
       1.94825907e-02, 8.63945950e-03, 8.10373109e-04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [22]:
TNGfilename = '/SFRMetallicityFromGasTNG100.hdf5'
with h5.File(data_dir+TNGfilename, "r") as f:
    MetalBins2         = f["MetalBins"][:]
    Obs_Lookbacktimes2 = f["Lookbacktimes"][:]
    BoxSfr2            = f["Sfr"][:]
    Redshifts2         = f["Redshifts"][:]

In [23]:
BoxSfr[1]-BoxSfr2[1]

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.52606517e-05, -1.52587891e-05,  0.00000000e+00,
        0.00000000e+00, -9.31322575e-10,  1.52587891e-05,  0.00000000e+00,
       -3.05175781e-05, -1.52587891e-05,  4.57763672e-05,  0.00000000e+00,
       -1.52587891e-05, -9.31322575e-10,  1.52597204e-05, -4.57754359e-05,
        4.57763672e-05,  0.00000000e+00, -3.05175781e-05,  1.52606517e-05,
       -1.52587891e-05,  1.52587891e-05,  0.00000000e+00, -1.52587891e-05,
        3.05175781e-05,  0.00000000e+00, -3.05175781e-05,  0.00000000e+00,
        1.52587891e-05,  0.00000000e+00, -1.52587891e-05,  4.57763672e-05,
       -3.05175781e-05,  1.52569264e-05,  0.00000000e+00, -3.05175781e-05,
        1.52587891e-05, -1.52587891e-05,  3.05175781e-05, -1.52587891e-05,
        1.52587891e-05,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [25]:
f1 = h5.File(data_dir+'/testfile.hdf5', 'r+') # open the file
data = f1['Sfr']       # load the data

In [33]:
data[3] = BoxSfr2[3]                      # assign new values to data

In [35]:
data[4]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [36]:
f1.close()                             # close the file

In [58]:
f1 = h5.File(data_dir+'/testfile.hdf5', 'r+') # open the file
data = f1['Sfr']       # load the data

In [59]:
data[3]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.25569183e-02, 1.22955529e+03, 1.66818397e+02, 1.53284260e+01,
       1.13097049e+01, 1.03234348e+01, 9.77201051e+00, 9.61273024e+00,
       9.49164883e+00, 9.92285927e+00, 1.02778060e+01, 1.07071474e+01,
       1.09266056e+01, 1.13967013e+01, 1.19514956e+01, 1.26565886e+01,
       1.31943894e+01, 1.38536936e+01, 1.46033194e+01, 1.53562769e+01,
       1.59991190e+01, 1.63683443e+01, 1.69056107e+01, 1.70176302e+01,
       1.70166471e+01, 1.67308770e+01, 1.63544514e+01, 1.63445470e+01,
       1.74806237e+01, 1.97745645e+01, 2.30537213e+01, 2.66671600e+01,
       3.21289043e+01, 4.23461230e+01, 6.25266229e+01, 9.45095807e+01,
       1.36583153e+02, 1.70330341e+02, 1.73198145e+02, 1.40700580e+02,
       8.39639589e+01, 2.69995552e+01, 4.34869385e+00, 5.62988281e-01,
       4.68750000e-02, 2.44140625e-04, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [61]:
n=2
m=10
data[n:m]

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        3.41320294e-03, 8.25346733e+02, 9.62495352e+01, 8.42699432e+00,
        6.24661177e+00, 5.63190995e+00, 5.22386167e+00, 5.21247358e+00,
        5.20240125e+00, 5.42136094e+00, 5.59039243e+00, 5.76143463e+00,
        5.94602399e+00, 6.34184584e+00, 6.37631335e+00, 6.87313527e+00,
        7.19019542e+00, 7.57035526e+00, 7.98460677e+00, 8.10980093e+00,
        8.50757586e+00, 8.69715312e+00, 9.20174132e+00, 9.04296158e+00,
        9.15850989e+00, 8.95789936e+00, 8.94647050e+00, 9.36962621e+00,
        1.01796050e+01, 1.24147153e+01, 1.51599211e+01, 1.67480736e+01,
        1.82274938e+01, 2.36075123e+01, 3.29552970e+01, 5.04705927e+01,
        7.28360985e+01, 8.88997891e+01, 8.65241195e+01, 6.60843469e+01,
        3.48332417e+01, 8.67355491e+00, 6.85058594e-01, 4.24804688e-02,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000

In [50]:
data[2] = BoxSfr2[2]

In [57]:
f1.close()

## 

In [28]:
#computes the lookback time at a given expansion factor (in Gyr h^-1) [this must be the driver]
def LookbackTime_a(a, is_flat):
    convert_in_Gyr = 3.085678e10 / 3.1536e7
    h = sim['hubble']
    HubbleConst =  67.8  #in km/sec/Mpc (at redshift 0)
    OmegaMatter =  sim['omega_0']
    OmegaLambda =  sim['omega_L']

    if type(a) == np.ndarray:
        if a.any() < 0.0:
            raise TypeError('The expansion factor provided is negative. This is not allowed')
        else:
            if a < 0.0:
                raise TypeError('The expansion factor provided is negative. This is not allowed')

        var1 = np.sqrt(OmegaLambda / OmegaMatter)
        fac1 = var1 + np.sqrt(1. + var1 * var1)
        var2 = np.sqrt(OmegaLambda / OmegaMatter * a**3.0)
        fac2 = var2 + np.sqrt(1. + var2 * var2)
        t_look = 2./(3. * np.sqrt(OmegaLambda) * HubbleConst) * np.log(fac1 / fac2)
        return t_look * convert_in_Gyr

In [31]:
def cosmology_init( self ):
    self.cosmo = CosmologicalFactors( my_h = self.hubbleparam, my_OmegaMatter = self.omega0, my_OmegaLambda = self.omegalambda )
    self.cosmo.SetLookbackTimeTable()
    return
def cosmology_get_lookback_time_from_a(a, is_flat=False, quicklist=False ):
    return (LookbackTime_a(a, is_flat) / h)

In [128]:
cosmology_get_lookback_time_from_a(time, is_flat=True )

In [ ]:
fname = "%s/output/snapdir_%03d/snap_%03d.%s.hdf5" % (run, snap, snap, "%d")
    for ifile in range( s.num_files ):
      with h5py.File(fname % ifile, "r") as f:

        pStars = f["PartType4"]

        Ages   = pStars["GFM_StellarFormationTime"][:]
        Masses = pStars["GFM_InitialMass"][:]
        Metals = pStars["GFM_Metallicity"][:]

In [13]:
data_dir = '/home/sashalvna/research/TNGdata/TNG100'
os.chdir(data_dir)

In [138]:
SFRs = []
metallicity_bins = np.logspace(-10, 0, 61) #metallicity bins

for ind in range(7): #ind, val in enumerate(snaps):
    
    f_Z = "fof_subhalo_tab_%03d.Subhalo.SubhaloGasMetallicitySfrWeighted.hdf5"%ind
    f_SFR = "fof_subhalo_tab_%03d.Subhalo.SubhaloSFR.hdf5"%ind
    
    #first download the data, if doesn't already exist
    if not os.path.isfile(f_Z):
        os.system("wget -nd -nc -nv -e robots=off -l 1 -r -A hdf5 --content-disposition --header='API-Key: e36226423a0cc5e62f2e553f39b44238' 'http://www.tng-project.org/api/%s/files/groupcat-%03d/?Subhalo=SubhaloGasMetallicitySfrWeighted'"%(names[i],ind))
    if not os.path.isfile(f_SFR):
        os.system("wget -nd -nc -nv -e robots=off -l 1 -r -A hdf5 --content-disposition --header='API-Key: e36226423a0cc5e62f2e553f39b44238' 'http://www.tng-project.org/api/%s/files/groupcat-%03d/?Subhalo=SubhaloSFR'"%(names[i],ind))

    #read in data from files
    data_Z = h5.File(f_Z, 'r')
    data_Sfr = h5.File(f_SFR, 'r')
    metallicities = data_Z['Subhalo']['SubhaloGasMetallicitySfrWeighted'][:]
    sfr = data_Sfr['Subhalo']['SubhaloSFR'][:]
    
    #bin metallicities and SFRs
    bin_ind = np.digitize(metallicities, bins=metallicity_bins)
    
    binned_sfr = np.zeros(len(metallicity_bins)-1)
    for j in np.unique(bin_ind):
        indices = np.where(bin_ind == j)[0]
        binned_sfr[j] = np.sum(sfr[indices])
    SFRs.append(binned_sfr)
    
SFRs = np.array(SFRs)